## ASHRAE 140-2023

Section 10: CASE HE210 - Realistic Weather Data

In [1]:
import OpenSimula as osm
import pandas as pd
import numpy as np

case_he210_dict = {
    "name": "Case HE210",
    "time_step": 3600,
    "n_time_steps": 2160,
    "initial_time": "01/01/2001 00:00:00",
    "simulation_file_met": "HEATING_WEATHER",
    "components": [
        {
            "type": "File_met",
            "name": "HEATING_WEATHER",
            "file_type": "WYEC2",
            "file_name": "../../mets/HE210W.WY2"
        },    
        {
            "type": "Material",
            "name": "Insulation",
            "conductivity": 0.01,
            "density": 1,
            "specific_heat": 1
        },
        {
            "type": "Material",
            "name": "Roof_Material",
            "conductivity": 0.0714,
            "density": 10,
            "specific_heat": 1000
        },
        {
            "type": "Construction",
            "name": "Adiabatic_Wall",
            "solar_alpha": [0,0],
            "lw_epsilon": [0,0],
            "materials": [
                "Insulation"
            ],
            "thicknesses": [
                1.0
            ]
        },
        {
            "type": "Construction",
            "name": "Roof_construction",
            "solar_alpha": [0,0],
            "lw_epsilon": [0,0],
            "materials": [
                "Roof_Material"
            ],
            "thicknesses": [
                0.01
            ]
        },
        {
            "type": "Space_type",
            "name": "constant_gain_space",
            "people_density": "0",
            "light_density": "0",
            "other_gains_density": "0",
            "other_gains_radiant_fraction": 0,
            "infiltration": "0"
        },
        {
            "type": "Building",
            "name": "Building",
            "azimuth": 0
        },
        {
            "type": "Space",
            "name": "spaces_1",
            "building": "Building",
            "spaces_type": "constant_gain_space",
            "floor_area": 48,
            "volume": 129.6,
            "furniture_weight": 0
        },
        {
            "type": "Building_surface",
            "name": "north_wall",
            "construction": "Adiabatic_Wall",
            "spaces": "spaces_1",
            "ref_point": [
                8,
                6,
                0
            ],
            "width": 8,
            "height": 2.7,
            "azimuth": 180,
            "altitude": 0,
            "h_cv": [
                0,
                0
            ]
        },
        {
            "type": "Building_surface",
            "name": "east_wall",
            "construction": "Adiabatic_Wall",
            "spaces": "spaces_1",
            "ref_point": [
                8,
                0,
                0
            ],
            "width": 6,
            "height": 2.7,
            "azimuth": 90,
            "altitude": 0,
             "h_cv": [
                0,
                0
            ]
        },
        {
            "type": "Building_surface",
            "name": "south_wall",
            "construction": "Adiabatic_Wall",
            "spaces": "spaces_1",
            "ref_point": [
                0,
                0,
                0
            ],
            "width": 8,
            "height": 2.7,
            "azimuth": 0,
            "altitude": 0,
             "h_cv": [
                0,
                0
            ]
        },
        {
            "type": "Building_surface",
            "name": "west_wall",
            "construction": "Adiabatic_Wall",
            "spaces": "spaces_1",
            "ref_point": [
                0,
                6,
                0
            ],
            "width": 6,
            "height": 2.7,
            "azimuth": -90,
            "altitude": 0,
             "h_cv": [
                0,
                0
            ]
        },
        {
            "type": "Building_surface",
            "name": "roof_wall",
            "construction": "Roof_construction",
            "spaces": "spaces_1",
            "ref_point": [
                0,
                0,
                2.7
            ],
            "width": 8,
            "height": 6,
            "azimuth": 0,
            "altitude": 90,
            "h_cv": [
                20,
                20
            ]
        },
        {
            "type": "Building_surface",
            "name": "floor_wall",
            "construction": "Adiabatic_Wall",
            "spaces": "spaces_1",
            "ref_point": [
                0,
                6,
                0
            ],
            "width": 8,
            "height": 6,
            "azimuth": 0,
            "altitude": -90,
            "h_cv": [
                0,
                0
            ]
        },
        {
            "type": "HVAC_DX_equipment",
            "name": "HVAC_equipment",
            "nominal_air_flow": 0.355,
            "nominal_heating_capacity": 10000,
            "nominal_heating_power": 12500,
            "indoor_fan_power": 200,
            "indoor_fan_operation": "CYCLING",
            "COP_expression": "F_load/(0.0080472574+0.87564457*F_load+0.29249943*F_load**2-0.17624156*F_load**3)",
        },
        {
            "type": "HVAC_DX_system",
            "name": "system",
            "spaces": "spaces_1",
            "equipment": "HVAC_equipment",
            "air_flow": 0.355,
            "outdoor_air_fraction": 0,
            "heating_setpoint": "20",
            "cooling_setpoint": "30",
            "system_on_off": "1"
        }
    ]
}


sim = osm.Simulation()
pro = sim.new_project("pro")
pro.read_dict(case_he210_dict)
pro.component("HEATING_WEATHER").set_location(39.833,-104.65,1650,-7*15)
pro.simulate()

Reading project data from dictonary
Reading completed.
Checking project: Case HE210
Checking completed.
Calculating solar direct shadows ...
Simulating Case HE210: ...


100%|██████████| 2160/2160 [00:02<00:00, 739.41step/s, n_iter=3]


In [3]:
system = pro.component("system")
sys_df=system.variable_dataframe()
sys_df["time_frac"] = sys_df["F_load"]/sys_df["efficiency_degradation"]
sys_df["draft_fan_power"] = sys_df["time_frac"]*50
consumo_tot = sys_df["power"].sum()*3600/1e9
consumo_fan = sys_df["indoor_fan_power"].sum()*3600/1e9
consumo_fuel = consumo_tot - consumo_fan

print("Furnace Load (GJ): ",sys_df["Q_sensible"].sum()*3600/1e9)
print("Furnace Input (GJ): ",consumo_fuel)
print("Fuel Concumption (m^3/s): ",(consumo_fuel*1000/38)/(90*3600*24))
print("Fans energy (kWh): ", (sys_df["draft_fan_power"].sum()+sys_df["indoor_fan_power"].sum())/1000 )

space_df = pro.component("spaces_1").variable_dataframe()
print("Mean zone temp. (ºC): ", space_df["temperature"].mean())
print("Max zone temp. (ºC): ", space_df["temperature"].max())
print("Min zone temp. (ºC): ", space_df["temperature"].min())



Furnace Load (GJ):  42.077259977178485
Furnace Input (GJ):  52.452936144982054
Fuel Concumption (m^3/s):  0.00017751291472067245
Fans energy (kWh):  291.40520080545593
Mean zone temp. (ºC):  20.0
Max zone temp. (ºC):  20.000000000000004
Min zone temp. (ºC):  19.999999999999996
